In [1]:
!pip install langchain langchain-community faiss-cpu sentence-transformers pymupdf python-docx
!pip install transformers accelerate


In [3]:
import os
from langchain_community.document_loaders import PyMuPDFLoader, Docx2txtLoader, TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline
from IPython.display import Markdown, display


d:\1st app\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
file_paths = [
    r"C:\Users\DAVESH KUMAR SAHU\Downloads\resume1.pdf"  
]
print("Files to load:", file_paths)


Files to load: ['C:\\Users\\DAVESH KUMAR SAHU\\Downloads\\resume1.pdf']


In [5]:
all_docs = []

for file_path in file_paths:
    ext = os.path.splitext(file_path)[1].lower()

    if ext == ".pdf":
        loader = PyMuPDFLoader(file_path)
    elif ext == ".docx":
        loader = Docx2txtLoader(file_path)
    elif ext == ".txt":
        loader = TextLoader(file_path)
    else:
        print(f"[!] Unsupported file type: {file_path}")
        continue

    docs = loader.load()
    all_docs.extend(docs)

print(f"✅ Total documents loaded: {len(all_docs)}")


✅ Total documents loaded: 1


In [6]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(all_docs)

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = FAISS.from_documents(chunks, embedding_model)

print(f"✅ Vector DB created with {len(chunks)} chunks.")


C:\Users\DAVESH KUMAR SAHU\AppData\Local\Temp\ipykernel_6260\1040686971.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


✅ Vector DB created with 5 chunks.


In [7]:
query = input("🔍 Ask something about the documents: ")

retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 3})
docs = retriever.get_relevant_documents(query)

print("Top matching document chunks:")
for i, doc in enumerate(docs):
    print(f"Chunk {i+1}:\n{doc.page_content}\n")



Top matching document chunks:
Chunk 1:
Technical Projects
Human Pose Estimation Using OpenCV
Oct 2024 – Jan 2025
Built a real-time pose tracking system using MediaPipe and OpenCV to detect 33 body landmarks for
gesture recognition and fitness applications.
ML Model for Handwritten Digit Recognition
Jun 2024 – Sep 2024
Achieved 95% accuracy using scikit-learn and CNNs on the MNIST dataset for digit classification.
Autonomous Robot for Smart Restaurant
Feb 2024 – Jun 2024

Chunk 2:
microcontrollers.
Skills
C, C++, Python, ROS2, OpenCV, Scikit-learn, TensorFlow, Embedded Systems, Arduino, Raspberry
Pi, Jetson Nano, Unity 3D, Blender, Git, Docker, Linux, IoT, Sensor Integration
Achievements & Certifications
• IEEE Idea Competition – 3rd Position for Energy Conservation (Mar 2022)
• Hackathon – AVESH, GEC Raipur (Mar 2023)
• Machine Learning Course – Udemy (Jul–Sep 2023)
• Backend Web Development – GDSC KIIT (Aug 2023)

Chunk 3:
Davesh Kumar Sahu
daveshkumarsahu496@gmail.com — +91 913192450

C:\Users\DAVESH KUMAR SAHU\AppData\Local\Temp\ipykernel_6260\355782741.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


In [8]:
qa_model = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=512
)

context = "\n".join([doc.page_content for doc in docs])
final_prompt = f"Answer the following question based on the text:\n\n{context}\n\nQuestion: {query}"

response = qa_model(final_prompt)[0]['generated_text']

display(Markdown(f"###  Final Answer:\n\n{response}"))


Device set to use cuda:0


###  Final Answer:

+91 9131924504